In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

In [2]:
import matplotlib as mpl
mpl.rcParams["axes.titlesize"] = 18

In [7]:
df = pd.read_csv('dataset.csv')
df.shape


(378661, 15)

In [8]:
NUM_VARS = ['usd_goal_real', 'usd_pledged_real', 'backers']
CAT_VARS  = ['category', 'main_category', 'country', 'currency']
TARGET = 'state'

In [9]:
import sklearn
sklearn.__version__

'0.19.1'

In [10]:
num_features = NUM_VARS
cat_features = CAT_VARS

In [12]:
X = df.loc[:, num_features + cat_features]
y = df[TARGET]
X.shape, y.shape

((378661, 7), (378661,))

In [13]:
for col in CAT_VARS:
    X[col] = X[col].astype('category').cat.codes

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((283995, 7), (283995,), (94666, 7), (94666,))

In [16]:
from sklearn.ensemble import RandomForestClassifier

In [17]:
m = RandomForestClassifier()

In [18]:
m.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [19]:
m.score(x_train, y_train), m.score(x_test, y_test)

(0.9699220056691139, 0.8568863161008176)

# Features engeering

In [22]:
np.random.randint(1,5)

2

# Random Forest

In [23]:
rf = RandomForestClassifier(n_estimators=20, oob_score=True)
rf.fit(x_train, y_train)

/Users/nguyenkhacbaoanh/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/nguyenkhacbaoanh/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [24]:
rf.oob_score_

0.8528424796211201

In [45]:
accuracy_score(y_test, rf.predict(x_test))

0.8648511609236684

In [33]:
#rf.estimators_

In [26]:
first_dt = rf.estimators_[0]
first_dt

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False,
            random_state=892980561, splitter='best')

In [27]:
first_dt.predict(x_test)

array([1., 3., 1., ..., 1., 0., 1.])

In [40]:
list(rf.classes_)

array(['canceled', 'failed', 'live', 'successful', 'suspended',
       'undefined'], dtype=object)

In [36]:
from sklearn.preprocessing import LabelEncoder

In [41]:
lb = LabelEncoder()
lb.fit(list(rf.classes_))
y_test_lb = lb.transform(y_test)

In [42]:
y_test_lb

array([1, 3, 1, ..., 0, 1, 0])

In [31]:
from sklearn.metrics import accuracy_score

In [43]:
preds = [accuracy_score(y_test_lb, m.predict(x_test)) for m in rf.estimators_]

In [44]:
np.mean(preds)

0.8091606278917457